In [7]:
import torch
import random
import math
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import networkx as nx

import sys



In [136]:
def initialize_graph():
    
    coordinates = pd.read_csv("vienna_dist.csv", header = None, sep=' ')
    coordinates.columns = ['coord1','coord2','dist']
    graph = nx.DiGraph()
    
    # add the rows to the graph for shortest path and distance calculations
    for _, row in coordinates.iterrows():
        graph.add_edge(row['coord1'], row['coord2'], weight=row['dist'])
        
    return graph


def precompute_shortest_path(graph, start_node, end_node):
    
    shortest_path = nx.shortest_path(graph, start_node, end_node)
    
    # TODO: distance need to be normalized afterwords
    shortest_path_length = sum(graph.get_edge_data(u, v)['weight'] 
                               for u, v in zip(shortest_path, shortest_path[1:]))
    
    return shortest_path, shortest_path_length
    

In [132]:
def get_distanceLL(lat1, lon1, lat2, lon2):
    
    R = 6371  # Radius of the Earth in kilometers
    
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

def get_NearestNodeLL(lat, lon, lats, lons):
    nearest = (-1, sys.float_info.max)
    for i in range(len(lats)):
        dist = get_distanceLL(lat, lon, lats[i], lons[i])
        if dist < nearest[1]:
            nearest = (i, dist)
    return nearest[0]


def get_depot_location(data_vienna):
    
    ll = (48.178808, 16.438460)
    lat = ll[0] / 180 * math.pi
    lon = ll[1] / 180 * math.pi
    lats = data_vienna['lats']
    lons = data_vienna['lons']
    depot = get_NearestNodeLL(lat, lon, lats, lons)
    depot_coordinates = np.array(data_vienna[data_vienna['id']==depot][['id','xcoords', 'ycoords']])

    return depot_coordinates

In [133]:

def get_customers_coordinates(data_vienna, batch_size, customers_count, depot):
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    
    # excluding depot id from the customers selection
    data_vienna = data_vienna[data_vienna['id']!= int(depot[0][0])]
    
    locations = []

    for _ in range(batch_size):
        
        sampled_customers = torch.multinomial(torch.tensor(data_vienna.id, dtype=torch.float), 
                                          num_samples=customers_count, replacement=True)
        
        sampled_location = data_vienna.loc[sampled_customers].reset_index(drop=True) 
        locations.append(sampled_location)
        
    # Create PyTorch tensors for the batched data
    locations_tensors = []
    
    for batch in locations:
        
        id_tensor = torch.tensor(batch['id'].values, dtype=torch.long)
        coords_tensor = torch.tensor(batch[['xcoords', 'ycoords']].values, dtype=torch.float)
        batch_tensor = torch.cat((id_tensor.unsqueeze(1), coords_tensor), dim=1)
        locations_tensors.append(batch_tensor)
        
    return torch.stack(locations_tensors)
        

In [226]:
def get_edges_attributes(batch_size, graph, depot, locations, V):
    
    # all customers ID inclusing depot
    edge_depot = torch.zeros((batch_size, 1, 1))
    edge_depot[:,:,:1] = depot[0][0]
    edge_data = torch.cat((edge_depot, locations[:,:,None,0]), dim=1)
    
    # generate edge index
    edges_index = []

    for i in range(V+1):
        for j in range(V+1):
            edges_index.append([i, j])
    edges_index = torch.LongTensor(edges_index)
    edges_index = edges_index.transpose(dim0=0,dim1=1)
    
    # generate nodes attributes
    edges_batch = []
    
    for batch in edge_data:
        edges = np.zeros((V+1, V+1, 1))
        for i, id1 in enumerate(batch):
            for j, id2 in enumerate(batch):
                _, distance = precompute_shortest_path(graph, int(id1), int(id2))
                edges[i][j][0] = distance

        edges = edges.reshape(-1, 1)
        edges_batch.append(torch.from_numpy(edges))
    
    return edges_index, torch.stack(edges_batch)

In [227]:
total_nodes = 16080

Lambda = 0.025 # request rate per min
dod = 0.5
horizon = 400
fDmean = 10
fDstd = 2.5
batch_size = 2

# static customer counts V = Lambda*horizon*(1-dod)/(dod+0.5)
V_static = int(Lambda*horizon*(1-dod)/(dod)+0.5)

# total customer count
V = int(Lambda*horizon/(dod) + 0.5)

# initialize the graph of vienna network
graph = initialize_graph()

# get the coordinates of customers
data_vienna = pd.read_csv('vienna_cordinates.csv')

depot = get_depot_location(data_vienna)

locations = get_customers_coordinates(data_vienna, batch_size, V, depot)

edges_index, edge_attributes = get_edges_attributes(batch_size, graph, depot, locations, V)
